In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [2]:
melbourne_file_path = '../input/melbourne-housing-snapshot/melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path)

In [3]:
iowa_file_path = '../input/home-data-for-ml-course/train.csv'
iowa_data = pd.read_csv(iowa_file_path)

# simple DecisionTreeRegressor model creation

In [4]:
# drops missing values
melbourne_data = melbourne_data.dropna(axis=0)

# target variable y
y = melbourne_data.Price

# features selected for judgement
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude']
X = melbourne_data[melbourne_features]

# validation

In [5]:
X, y = X.values, y.values
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 42, shuffle=True, test_size=0.10)

In [6]:
def training_crossval_models(X,Y):
    best_models = []
    moyenne = 0
    liste_predictions = []
    liste_indexes = []
    for i in tqdm(range(100)):
        y_preds_class = None
        
        kf = KFold(n_splits=6, shuffle=True, random_state=np.random.randint(9,999))
        for j, (train_index, test_index) in enumerate(kf.split(X)):
            X_train, X_val = X[train_index], X[test_index]
            Y_train, Y_val = Y[train_index], Y[test_index]
            model = RandomForestRegressor(random_state=42)
            model.fit(X_train, Y_train)
            predictions = model.predict(X_val)
            liste_predictions.append(predictions)
            liste_indexes.append(test_index)
    return liste_predictions, liste_indexes


def correction_data(X, Y, liste_predictions, liste_indexes, topk=10):
    dico = {}
    for i in range(len(liste_predictions)):
        liste_prediction = liste_predictions[i]
        liste_index = liste_indexes[i]
        for j in range(len(liste_index)):
            if liste_index[j] not in dico:
                dico[liste_index[j]] = [liste_prediction[j]]
            else:
                dico[liste_index[j]] += [liste_prediction[j]]

    entropy = {}
    values_mean = {}
    for i in dico:
        liste = []
        for value_prediction in dico[i]:
            liste.append((value_prediction-Y[i])**2)
        liste = np.array(liste)
        entropy_ = liste
        entropy[i] = np.log(np.mean(entropy_))- 2*np.std(dico[i])
        values_mean[i] = np.mean(dico[i])
    liste = list(entropy.items())
    keys, values = [], []
    for i,j in liste:
        keys.append(i)
        values.append(j)
        
    indexes_sorted = np.argsort(values)[::-1][:topk]
    keys = np.array(keys)[indexes_sorted]
    values = np.array(values)[indexes_sorted]
    for i,key_index in enumerate(keys):
        Y[key_index] = values_mean[key_index]
        
    return Y

In [7]:
for i in range(150):
    model = RandomForestRegressor(random_state=42)
    model.fit(train_X, train_y)
    val_predictions = model.predict(val_X)
    val_mae = np.sqrt(mean_absolute_error(val_y,val_predictions))
    print("Validation {} MAE for best value of max_leaf_nodes: {:,.5f}".format(i, val_mae))
    
    liste_predictions, liste_indexes = training_crossval_models(train_X, train_y)
    train_y = correction_data(train_X,train_y, liste_predictions, liste_indexes)

# log(x) -2*std HAUT POTENTIEL (sans overflow)

  0%|          | 0/100 [00:00<?, ?it/s]

Validation 0 MAE for best value of max_leaf_nodes: 418.03501


  1%|          | 1/100 [00:08<14:39,  8.89s/it]

  2%|▏         | 2/100 [00:17<14:34,  8.92s/it]

  3%|▎         | 3/100 [00:26<14:26,  8.93s/it]

...

 22%|██▏       | 22/100 [03:18<11:44,  9.03s/it]

 23%|██▎       | 23/100 [03:27<11:35,  9.04s/it]

 24%|██▍       | 24/100 [03:36<11:25,  9.02s/it]

 25%|██▌       | 25/100 [03:45<11:15,  9.01s/it]

 26%|██▌       | 26/100 [03:54<11:07,  9.02s/it]

In [ ]:
# refaire la formule basé sur l'AIC et la BIC
# https://fr.wikipedia.org/wiki/Critère_d'information_d'Akaike

In [ ]:
log(x)-std est la transformation sans overflow de x/exp(std) 
plus std est petit et plus x est grand plus il doit etre corrigé
TESTER log(x)-std**2 (parce que x au carré)
trouver la formule avec brice

In [ ]:
for i in range(150):
    model = RandomForestRegressor(random_state=42)
    model.fit(train_X, train_y)
    val_predictions = model.predict(val_X)
    val_mae = np.sqrt(mean_absolute_error(val_y,val_predictions))
    print("Validation {} MAE for best value of max_leaf_nodes: {:,.5f}".format(i, val_mae))
    
    liste_predictions, liste_indexes = training_crossval_models(train_X, train_y)
    train_y = correction_data(train_X,train_y, liste_predictions, liste_indexes)
    
# x/exp(std) NON

In [ ]:
for i in range(150):
    model = RandomForestRegressor(random_state=42)
    model.fit(train_X, train_y)
    val_predictions = model.predict(val_X)
    val_mae = np.sqrt(mean_absolute_error(val_y,val_predictions))
    print("Validation {} MAE for best value of max_leaf_nodes: {:,.5f}".format(i, val_mae))
    
    liste_predictions, liste_indexes = training_crossval_models(train_X, train_y)
    train_y = correction_data(train_X,train_y, liste_predictions, liste_indexes)
    
# x*exp(x)/std NON

In [ ]:
for i in range(150):
    model = RandomForestRegressor(random_state=42)
    model.fit(train_X, train_y)
    val_predictions = model.predict(val_X)
    val_mae = np.sqrt(mean_absolute_error(val_y,val_predictions))
    print("Validation {} MAE for best value of max_leaf_nodes: {:,.5f}".format(i, val_mae))
    
    liste_predictions, liste_indexes = training_crossval_models(train_X, train_y)
    train_y = correction_data(train_X,train_y, liste_predictions, liste_indexes)

# exp(-x)/x/std NON

In [ ]:
# laisser tourner 15 epoques pour voir ce que ça donne avec ça
# remettre std(y) car cest ce que lon cherche donc np.mean(MAE)/np.std(ŷ)
